# Advanced Lane Line Finding
Steps to be applied:
1. Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
2. Apply a distortion correction to raw images.
3. Use color transforms, gradients, etc., to create a thresholded binary image.
4. Apply a perspective transform to rectify binary image ("birds-eye view").
5. Detect lane pixels and fit to find the lane boundary.
6. Determine the curvature of the lane and vehicle position with respect to center.
7. Warp the detected lane boundaries back onto the original image.
8. Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

In [6]:
# First import necessary libraries
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import glob
import cv2
from datetime import datetime

def now():
    return datetime.now().strftime("%d_%m_%Y-%H_%M_%S_%f")

## Step 1: Camera calibration and distortion coefficients

In [7]:
def processCalibrationImages(path_to_cal_images, nx, ny, save = False):
    ## This function processes the calibration images by finding the chessboard corners in each calibration image
    ## and appending them to an imagepoint list.
    ## Inputs: 
    ## - path_to_cal_images: the filepath to the calibration images. 
    ##                      The filepath can include wildcards if the images are numbered
    ## - nx, ny: the number of inner chessboard corners in x and y direction
    ## - save: if this flag is used, the chessboard corners are drawn on the calibration image and saved
    ## Output: 
    ## - imgpointlist: the list of found image points 
    ## - objpointlist: the list of object points that correspond to the found image points
    
    #Load the calibration image names
    cal_images = glob.glob(path_to_cal_images)

    #Initialize object and image point list and object point array
    imgpointlist = []
    objpointlist = []
    objpoints = np.zeros((nx*ny,3), np.float32)
    objpoints[:,:2] = np.mgrid[0:nx,0:ny].T.reshape(-1,2)

    #Loop through each calibration image name
    for cal_img_name in cal_images:
        #Load the image using openCV
        image = cv2.imread(cal_img_name)
        #Convert to gray
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)

        # If corners are found
        if ret == True:
            #Add corners to image point list and append object point list
            imgpointlist.append(corners)
            objpointlist.append(objpoints)
            # If function is called with save-flag
            if save == True:
                #Draw the corners and save the images (just for fun)
                cv2.drawChessboardCorners(image, (nx, ny), corners, ret)
                cv2.imwrite('output_images/'+'Chessboard_'+cal_img_name.split('\\')[1], image)
                
    return imgpointlist, objpointlist

## Step 2: Apply distortion correction to raw images

In [8]:
def undistort_image(image, imgpointlist, objpointlist, save = False):
    ## This function takes in an image and undistorts it based on a given object and image point list.
    ## The function calculates the camera matrix and distortion coefficients. These are then used
    ## to undistort the given image
    ## Inputs: 
    ## - image: the image that will be undistored 
    ## - imgpointlist: the list of found image points 
    ## - objpointlist: the list of object points that correspond to the image points
    ## - save: if this flag is used, the undistorted image is saved
    ## Output: 
    ## - undist: the undistorted image
    
    #Step 1: Determine camera matrix and distortion coefficients
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpointlist, imgpointlist, image.shape[1::-1], None, None)
    #Step 2: Apply camera matrix and disortion coefficients to input image
    undist = cv2.undistort(image, mtx, dist, None, mtx)
    
    if save == True:
        cv2.imwrite('output_images/undistorted_image_'+now()+'.jpg', undist)
        
    return undist

## Step 3: Creating a thresholded binary image

In [207]:
#############################
## Functions for plotting ##
#############################

def plot_2(img1, img2, title1='Image 1', title2='Image 2', cmap=None):
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(18,6))
    ax1.set_title(title1, fontsize=20)
    ax1.imshow(img1, cmap=cmap)
    ax2.set_title(title2, fontsize=20)
    ax2.imshow(img2, cmap=cmap)
    plt.subplots_adjust(top=0.97, bottom=0.03, left=0.03, right=0.97)
    
def plot_3(img1, img2, img3, title1='Image 1', title2='Image 2', title3='Image3', cmap=None):
    f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(22, 6))
    ax1.imshow(img1, cmap=cmap)
    ax1.set_title(title1, fontsize=20)
    ax2.imshow(img2, cmap=cmap)
    ax2.set_title(title2, fontsize=20)
    ax3.imshow(img3, cmap=cmap)
    ax3.set_title(title3, fontsize=20)
    plt.subplots_adjust(top=0.97, bottom=0.03, left=0.03, right=0.97)
    
def plot_4(img1, img2, img3, img4, title1='Image 1', title2='Image 2', title3='Image3', title4='Image4', cmap=None):
    f, (axtop, axbot) = plt.subplots(2, 2, figsize=(18, 9))
    axtop[0].imshow(img1, cmap=cmap)
    axtop[0].set_title(title1, fontsize=20)
    axtop[1].imshow(img2, cmap=cmap)
    axtop[1].set_title(title2, fontsize=20)
    axbot[0].imshow(img3, cmap=cmap)
    axbot[0].set_title(title3, fontsize=20)
    axbot[1].imshow(img4, cmap=cmap)
    axbot[1].set_title(title4, fontsize=20)
    plt.subplots_adjust(top=0.97, bottom=0.03, left=0.03, right=0.97)
    
def bgr2rgb(image):
    return cv2.cvtColor(image,cv2.COLOR_BGR2RGB)

#############################
## Functions for gradients ##
#############################

def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0, 255)):
    # Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Apply cv2.Sobel()
    if orient == 'x':
        sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    else:
        sobel = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the output from cv2.Sobel()
    absolute = np.absolute(sobel)
    # Scale the result to an 8-bit range (0-255)
    scale = np.uint8(255*absolute/np.max(absolute))
    # Apply lower and upper thresholds
    binary_output = np.zeros_like(scale)
    binary_output[(scale >= np.min(thresh)) & (scale <= np.max(thresh))] = 1
    # Create binary_output
    return binary_output, scale

def mag_threshold(img, sobel_kernel=3, thresh=(0, 255)):
    
    # Apply the following steps to img
    # 1) Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # 2) Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray, cv2.CV_64F,1,0,ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F,0,1,ksize=sobel_kernel)
    # 3) Calculate the magnitude
    magnitude = np.sqrt(np.square(sobelx) + np.square(sobely))
    # 4) Scale to 8-bit (0 - 255) and convert to type = np.uint8
    scaled = np.uint8(255/np.max(magnitude) * magnitude)
    # 5) Create a binary mask where mag thresholds are met
    binary_output = np.zeros_like(scaled)
    binary_output[(scaled >= np.min(thresh)) & (scaled <= np.max(thresh))] = 1
    # 6) Return this mask as your binary_output image
    return binary_output, scaled

def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Apply the following steps to img
    # 1) Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # 2) Take the gradient in x and y separately
    # 3) Take the absolute value of the x and y gradients
    abs_sobelx = np.absolute(cv2.Sobel(gray, cv2.CV_64F,1,0,ksize=sobel_kernel))
    abs_sobely = np.absolute(cv2.Sobel(gray, cv2.CV_64F,0,1,ksize=sobel_kernel))
    # 4) Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient
    direction = np.arctan2(abs_sobely, abs_sobelx)
    # 5) Create a binary mask where direction thresholds are met
    binary_output = np.zeros_like(direction)
    binary_output[(direction >= np.min(thresh)) & (direction <= np.max(thresh))] = 1
    # 6) Return this mask as your binary_output image
    return binary_output, direction


####################################
## Functions for color thresholds ##
####################################

def hls_select(img, thresh=(0, 255)):
    # 1) Convert to HLS color space
    hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    # 2) Apply a threshold to the S channel
    S = hls[:,:,2]
    # 3) Return a binary image of threshold result
    binary_output = np.zeros_like(S)
    binary_output[(S > np.min(thresh)) & (S <= np.max(thresh))] = 1
    return binary_output, S


##########################
## Step 3 Main Function ##
##########################

def create_binary(image, ksize=3, gradient_thresh=(0,255), mag_thresh=(0,255), dir_thresh=(0, np.pi/2), 
                  color_thresh = (0,255), save=False):

    # Apply each of the gradient threshold functions
    gradxb, gradx = abs_sobel_thresh(image, 'x', ksize, gradient_thresh)
    gradyb, grady = abs_sobel_thresh(image, 'y', ksize, gradient_thresh)
    mag_binary, mag = mag_threshold(image, ksize, mag_thresh)
    dir_binary, direct = dir_threshold(image, ksize, dir_thresh)
    #Combine gradient thresholds into one gradient binary
    gradients = np.zeros_like(dir_binary)
    gradients[((gradxb == 1) & (gradyb == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1
    
    #Apply color thresholds: In this implementation, the S channel of the image is used
    colors, S = hls_select(image, color_thresh)
    
    # Create the result image:
    # Wherever the gradient binary OR the color binary is activated (i.e. the pixel value = 1)
    # the result binary should be activataed as well
    resultb = np.zeros_like(gradients)
    resultb[(gradients == 1) | (colors == 1)] = 1
    
    # If the picture should be saved:
    if save == True:
        # Create plot combining the input image, the colored result binary and the result binary
        resultcombined = np.dstack((np.zeros_like(gradients), gradients, colors))
        plot_3(bgr2rgb(image), resultcombined, resultb, "Original","Combined Result Colored ", "Combined Result Binary", 'gray')
        plt.savefig('output_images/Color_Gradient_Threshold_'+now()+'.jpg')
        cv2.imwrite('output_images/binary_'+now()+'.jpg', resultb)
    return resultb#, gradients, colors, gradxb, gradyb, mag_binary, dir_binary



## Step 4: Perspective transform

In [10]:
def warp_image(img, src, dest, save=False):
    imshape = img.shape[1::-1]
    #use cv2.getPerspectiveTransform() to get M, the transform matrix
    M = cv2.getPerspectiveTransform(src, dest)
    #use cv2.warpPerspective() to warp your image to a top-down view
    warped = cv2.warpPerspective(img, M, imshape, flags=cv2.INTER_LINEAR)
    
    if save == True:
        imgdraw = np.copy(img)
        img = cv2.drawContours()
        plot_2(bgr2rgb(img), warped, "Original","Warped", 'gray')
        plt.savefig('output_images/Warped_'+now()+'.jpg')
    
    return warped

## Step 5: Detect lane pixels and fit to find the lane boundary

In [18]:
def get_activated_pixels(img):
    # Grab activated pixels
    nonzero = img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    return nonzerox, nonzeroy

def get_lane_pixels_from_indices(img, left_lane_inds, right_lane_inds):
    nonzerox, nonzeroy = get_activated_pixels(img)
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    return leftx, lefty, rightx, righty


def fit_poly(leftx, lefty, rightx, righty, imgshape):
    coef_left = np.polyfit(lefty, leftx, 2)
    coef_right = np.polyfit(righty, rightx, 2)
    poly_y_left = np.linspace(0, imgshape[0]-1, imgshape[0])
    poly_y_right = np.linspace(0, imgshape[0]-1, imgshape[0])
    try:
        poly_x_left = coef_left[0]*poly_y_left**2 + coef_left[1]*poly_y_left + coef_left[2]
        poly_x_right = coef_right[0]*poly_y_right**2 + coef_right[1]*poly_y_right + coef_right[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        poly_x_left = 1*poly_y_left**2 + 1*poly_y_left
        poly_x_right = 1*poly_y_right**2 + 1*poly_y_right
        coef_left, coef_right = []
    return coef_left, coef_right, poly_x_left, poly_y_left, poly_x_right, poly_y_right

def get_indices_from_poly(coef_left, coef_right, nonzerox, nonzeroy, margin = 100):
    ### TO-DO: Set the area of search based on activated x-values ###
    ### within the +/- margin of our polynomial function ###
    ### Hint: consider the window areas for the similarly named variables ###
    ### in the previous quiz, but change the windows to our new search area ###
    left_x_options =  coef_left[0]*nonzeroy**2 + coef_left[1]*nonzeroy + coef_left[2]
    right_x_options = coef_right[0]*nonzeroy**2 + coef_right[1]*nonzeroy + coef_right[2]
    left_lane_inds = (nonzerox > left_x_options - margin) & (nonzerox < left_x_options + margin)
    right_lane_inds = (nonzerox > right_x_options - margin) & (nonzerox < right_x_options + margin)
    return left_lane_inds, right_lane_inds
    

def get_indices_from_window(img, nonzerox, nonzeroy, nwindows=9, margin=100, minpix=50):
    bottom_half = img[img.shape[0]//2:,:]
    histogram = np.sum(bottom_half, axis=0)
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(img.shape[0]//nwindows)
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Create an output image to draw on and visualize the result
    #out_img = np.dstack((binary_warped, binary_warped, binary_warped))

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = img.shape[0] - (window+1)*window_height
        win_y_high = img.shape[0] - window*window_height
        ### TO-DO: Find the four below boundaries of the window ###
        win_xleft_low = leftx_current - margin  # Update this
        win_xleft_high = leftx_current + margin  # Update this
        win_xright_low = rightx_current - margin # Update this
        win_xright_high = rightx_current + margin  # Update this
        
        
        # Draw the windows on the visualization image
        #cv2.rectangle(out_img,(win_xleft_low,win_y_low), (win_xleft_high,win_y_high),(0,255,0), 2) 
        #cv2.rectangle(out_img,(win_xright_low,win_y_low), (win_xright_high,win_y_high),(0,255,0), 2) 

        ### TO-DO: Identify the nonzero pixels in x and y within the window ###
        good_left_inds = np.nonzero((nonzerox >= win_xleft_low) & (nonzerox <= win_xleft_high) & (nonzeroy >= win_y_low) & (nonzeroy <= win_y_high))[0]
        good_right_inds = np.nonzero((nonzerox >= win_xright_low) & (nonzerox <= win_xright_high) & (nonzeroy >= win_y_low) & (nonzeroy <= win_y_high))[0]

        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)

        ### TO-DO: If you found > minpix pixels, recenter next window ###
        ### (`right` or `leftx_current`) on their mean position ###
        if len(good_left_inds) > minpix:
            leftx_current = int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:
            rightx_current = int(np.mean(nonzerox[good_right_inds]))

    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)
    
    return left_lane_inds, right_lane_inds

def draw_pixels(img, leftx, lefty, rightx, righty):
    out_img = np.dstack((img, img, img))
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]
    return out_img

def draw_poly(poly_x_left, poly_y_left, poly_x_right, poly_y_right):
    plt.plot(poly_x_left, poly_y_left, color='yellow')
    plt.plot(poly_x_right, poly_y_right, color='yellow')

def visualize_lane(img, leftx, lefty, rightx, righty, poly_x_left, poly_y_left, poly_x_right, poly_y_right, margin = 100):
    ## Visualization ##
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((img, img, img))*255
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]
    
    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([poly_x_left-margin, poly_y_left]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([poly_x_left+margin, 
                              poly_y_left])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([poly_x_right-margin, poly_y_right]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([poly_x_right+margin, 
                              poly_y_right])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    
    ## End visualization steps ##
    return result

def visualize_lane2(img, leftx, lefty, rightx, righty, poly_x_left, poly_y_left, poly_x_right, poly_y_right, margin = 100):
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(img).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([poly_x_left, poly_y_left]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([poly_x_right, poly_y_right])))])
    pts = np.hstack((pts_left, pts_right))

    
    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    
    #Draw the pixels on the lane
    color_warp[lefty, leftx] = [255, 0, 0]
    color_warp[righty, rightx] = [0, 0, 255]
    
    return color_warp



## Step 6 Curvature and lane position

In [41]:
def measure_curvature_and_laneposition(poly_y_left, poly_y_right, poly_x_left, poly_x_right, ym_per_pix, xm_per_pix, imgshape):
    '''
    Calculates the curvature of polynomial functions in meters.
    '''
    coef_left_real = np.polyfit(poly_y_left*ym_per_pix, poly_x_left*xm_per_pix, 2)
    coef_right_real = np.polyfit(poly_y_right*ym_per_pix, poly_x_right*xm_per_pix, 2)
    
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max([poly_y_left*ym_per_pix, poly_y_right*ym_per_pix])
    
    ##### TO-DO: Implement the calculation of R_curve (radius of curvature) #####
    left_curverad = ((1 + (2*coef_left_real[0]*y_eval + coef_left_real[1])**2)**1.5) / np.absolute(2*coef_left_real[0])  ## Implement the calculation of the left line here
    right_curverad = ((1 + (2*coef_right_real[0]*y_eval + coef_right_real[1])**2)**1.5) / np.absolute(2*coef_right_real[0])  ## Implement the calculation of the right line here
    
    mean_curverad = np.average([left_curverad, right_curverad])
    
    lane_center_reference = (imgshape[1]-1)//2 * xm_per_pix
    y_lane_center = (imgshape[0]-1) * ym_per_pix
    x_left =  coef_left_real[0]*y_lane_center**2 + coef_left_real[1]*y_lane_center + coef_left_real[2]
    x_right = coef_right_real[0]*y_lane_center**2 + coef_right_real[1]*y_lane_center + coef_right_real[2]
    lane_position = np.average([x_left, x_right]) - lane_center_reference #positive value means right of center
    
    return mean_curverad, lane_position

## Main Section

In [204]:
### MAIN ###

def processimage(image):
    global imgp
    global objp
    global coef_left
    global coef_right
    ##Step 2: Apply distortion correction to undistort images
    ##This step will be step 1 for the video data since this is
    ##the first step that is actually performed on the raw images
    undistort = undistort_image(image, imgp, objp)

    ##Step 3: Creating a thresholded binary image
    ##This step takes the undistorted image (frame) and creates a binary image
    ##based on different thresholds for gradients and colors
    binary = create_binary(undistort, ksize=9, gradient_thresh=(60,200), mag_thresh=(60,200), dir_thresh=(0.7,1.3), 
                           color_thresh=(120,255), save=False)

    ##Step 4: Warping the image
    ##In this step, the image is warped to a birds-eye-view
    bottomleft=(192, 720)
    topleft=(576, 460)
    topright=(707, 460)
    bottomright=(1135, 720)
    inside = 300
    imshape = image.shape[1::-1]
    src = np.float32([bottomleft, topleft, topright, bottomright])
    #destination points for warp
    dst = np.float32([[inside, imshape[1]], [inside, 0], [imshape[0]-inside, 0], [imshape [0]-inside, imshape[1]]])
    binary_warped = warp_image(binary, src, dst)

    # Step 5: find the lane pixels
    #Get activated cells
    nonzerox,nonzeroy = get_activated_pixels(binary_warped)
    
    if (coef_left != []) & (coef_right != []):
        #Apply polynom method
        left_lane_inds, right_lane_inds = get_indices_from_poly(coef_left, coef_right, nonzerox, nonzeroy, margin = 100)
    else:
        #Apply window method
        left_lane_inds, right_lane_inds = get_indices_from_window(binary_warped, nonzerox, nonzeroy, nwindows=9, margin=100, minpix=50)
        
    
    #Get lane pixels from indices
    leftx, lefty, rightx, righty = get_lane_pixels_from_indices(binary_warped, left_lane_inds, right_lane_inds)
    
    #Apply polyfit to get new values for coef_left, coef_right
    coef_left, coef_right, poly_x_left, poly_y_left, poly_x_right, poly_y_right = fit_poly(leftx, lefty, rightx, righty, binary_warped.shape)  
    
    #Visualize result
    result = visualize_lane2(binary_warped, leftx, lefty, rightx, righty, poly_x_left, poly_y_left, poly_x_right, poly_y_right, margin = 100)

    ##Step 6:determine curvature
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    curvature, laneposition = measure_curvature_and_laneposition(poly_y_left, poly_y_right, poly_x_left, poly_x_right, ym_per_pix, xm_per_pix, result.shape)

    
    ##Step7: unwarp
    unwarped = warp_image(result, dst, src).astype('uint8')
    
    #Step 8: Plot on original
    final_result = cv2.addWeighted(undistort, 1, unwarped, 0.3, 0)
    
    cv2.putText(final_result, "Current curvature: {:.2f} meters".format(curvature), (50, 50), cv2.FONT_HERSHEY_PLAIN, 2, (255,255,255))
    cv2.putText(final_result, "Position in lane: {:.2f} meters right of center".format(laneposition), (50, 100), cv2.FONT_HERSHEY_PLAIN, 2, (255,255,255))
    
    
    return final_result



In [13]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML


In [209]:
first_output = 'project_video_out.mp4'
clip1 = VideoFileClip('project_video.mp4')#.subclip(20,25)

#Step 1: Canera Calibration
#Determine object points and img points from calibration images
#the calibration images have a (9, 6) inner chessboard
imgp, objp = processCalibrationImages('camera_cal/calibration*.jpg', 9, 6)

coef_left = []
coef_right = []

clip_output = clip1.fl_image(processimage)
%time clip_output.write_videofile(first_output, audio=False)

                                                                                                                       
t:  38%|████████████████████████▊                                         | 47/125 [5:56:13<01:26,  1.11s/it, now=None]
                                                                                                                       
t:  21%|██████████████▌                                                     | 6/28 [5:49:01<00:21,  1.01it/s, now=None]

t:   0%|                                                                            | 0/1260 [00:00<?, ?it/s, now=None]

Moviepy - Building video project_video_out.mp4.
Moviepy - Writing video project_video_out.mp4





t:   0%|                                                                    | 2/1260 [00:01<11:56,  1.76it/s, now=None]

t:   0%|▏                                                                   | 3/1260 [00:02<15:50,  1.32it/s, now=None]

t:   0%|▏                                                                   | 4/1260 [00:03<18:18,  1.14it/s, now=None]

t:   0%|▎                                                                   | 5/1260 [00:04<20:12,  1.03it/s, now=None]

t:   0%|▎                                                                   | 6/1260 [00:05<21:21,  1.02s/it, now=None]

t:   1%|▍                                                                   | 7/1260 [00:06<22:15,  1.07s/it, now=None]

t:   1%|▍                                                                   | 8/1260 [00:08<22:47,  1.09s/it, now=None]

t:   1%|▍                                                                   | 9/1260 [00:09<23:04,  1.11s/it, now=None]

t:   1%|▌                     

t:   5%|███▌                                                               | 66/1260 [01:22<24:41,  1.24s/it, now=None]

t:   5%|███▌                                                               | 67/1260 [01:23<24:29,  1.23s/it, now=None]

t:   5%|███▌                                                               | 68/1260 [01:25<24:22,  1.23s/it, now=None]

t:   5%|███▋                                                               | 69/1260 [01:26<24:29,  1.23s/it, now=None]

t:   6%|███▋                                                               | 70/1260 [01:27<24:16,  1.22s/it, now=None]

t:   6%|███▊                                                               | 71/1260 [01:28<24:02,  1.21s/it, now=None]

t:   6%|███▊                                                               | 72/1260 [01:30<26:18,  1.33s/it, now=None]

t:   6%|███▉                                                               | 73/1260 [01:31<27:28,  1.39s/it, now=None]

t:   6%|███▉                    

t:  10%|██████▊                                                           | 130/1260 [02:39<22:28,  1.19s/it, now=None]

t:  10%|██████▊                                                           | 131/1260 [02:41<23:39,  1.26s/it, now=None]

t:  10%|██████▉                                                           | 132/1260 [02:42<24:12,  1.29s/it, now=None]

t:  11%|██████▉                                                           | 133/1260 [02:44<23:59,  1.28s/it, now=None]

t:  11%|███████                                                           | 134/1260 [02:45<23:47,  1.27s/it, now=None]

t:  11%|███████                                                           | 135/1260 [02:46<23:21,  1.25s/it, now=None]

t:  11%|███████                                                           | 136/1260 [02:47<22:45,  1.21s/it, now=None]

t:  11%|███████▏                                                          | 137/1260 [02:48<22:30,  1.20s/it, now=None]

t:  11%|███████▏                

...

t:  81%|████████████████████████████████████████████████████▉            | 1026/1260 [19:54<04:28,  1.15s/it, now=None]

t:  82%|████████████████████████████████████████████████████▉            | 1027/1260 [19:55<04:28,  1.15s/it, now=None]

t:  82%|█████████████████████████████████████████████████████            | 1028/1260 [19:56<04:26,  1.15s/it, now=None]

t:  82%|█████████████████████████████████████████████████████            | 1029/1260 [19:57<04:25,  1.15s/it, now=None]

t:  82%|█████████████████████████████████████████████████████▏           | 1030/1260 [19:59<04:26,  1.16s/it, now=None]

t:  82%|█████████████████████████████████████████████████████▏           | 1031/1260 [20:00<04:26,  1.16s/it, now=None]

t:  82%|█████████████████████████████████████████████████████▏           | 1032/1260 [20:01<04:24,  1.16s/it, now=None]

t:  82%|█████████████████████████████████████████████████████▎           | 1033/1260 [20:02<04:23,  1.16s/it, now=None]

t:  82%|████████████████████████

t:  87%|████████████████████████████████████████████████████████▏        | 1090/1260 [21:08<03:15,  1.15s/it, now=None]

t:  87%|████████████████████████████████████████████████████████▎        | 1091/1260 [21:09<03:14,  1.15s/it, now=None]

t:  87%|████████████████████████████████████████████████████████▎        | 1092/1260 [21:10<03:13,  1.15s/it, now=None]

t:  87%|████████████████████████████████████████████████████████▍        | 1093/1260 [21:11<03:11,  1.15s/it, now=None]

t:  87%|████████████████████████████████████████████████████████▍        | 1094/1260 [21:12<03:10,  1.15s/it, now=None]

t:  87%|████████████████████████████████████████████████████████▍        | 1095/1260 [21:14<03:09,  1.15s/it, now=None]

t:  87%|████████████████████████████████████████████████████████▌        | 1096/1260 [21:15<03:08,  1.15s/it, now=None]

t:  87%|████████████████████████████████████████████████████████▌        | 1097/1260 [21:16<03:07,  1.15s/it, now=None]

t:  87%|████████████████████████

t:  92%|███████████████████████████████████████████████████████████▌     | 1154/1260 [22:22<02:03,  1.17s/it, now=None]

t:  92%|███████████████████████████████████████████████████████████▌     | 1155/1260 [22:23<02:01,  1.16s/it, now=None]

t:  92%|███████████████████████████████████████████████████████████▋     | 1156/1260 [22:24<02:00,  1.16s/it, now=None]

t:  92%|███████████████████████████████████████████████████████████▋     | 1157/1260 [22:25<01:59,  1.16s/it, now=None]

t:  92%|███████████████████████████████████████████████████████████▋     | 1158/1260 [22:26<01:58,  1.16s/it, now=None]

t:  92%|███████████████████████████████████████████████████████████▊     | 1159/1260 [22:28<01:57,  1.16s/it, now=None]

t:  92%|███████████████████████████████████████████████████████████▊     | 1160/1260 [22:29<01:56,  1.16s/it, now=None]

t:  92%|███████████████████████████████████████████████████████████▉     | 1161/1260 [22:30<01:56,  1.18s/it, now=None]

t:  92%|████████████████████████

t:  97%|██████████████████████████████████████████████████████████████▊  | 1218/1260 [23:36<00:48,  1.15s/it, now=None]

t:  97%|██████████████████████████████████████████████████████████████▉  | 1219/1260 [23:37<00:47,  1.15s/it, now=None]

t:  97%|██████████████████████████████████████████████████████████████▉  | 1220/1260 [23:38<00:45,  1.15s/it, now=None]

t:  97%|██████████████████████████████████████████████████████████████▉  | 1221/1260 [23:39<00:44,  1.15s/it, now=None]

t:  97%|███████████████████████████████████████████████████████████████  | 1222/1260 [23:40<00:43,  1.15s/it, now=None]

t:  97%|███████████████████████████████████████████████████████████████  | 1223/1260 [23:42<00:42,  1.15s/it, now=None]

t:  97%|███████████████████████████████████████████████████████████████▏ | 1224/1260 [23:43<00:41,  1.14s/it, now=None]

t:  97%|███████████████████████████████████████████████████████████████▏ | 1225/1260 [23:44<00:40,  1.14s/it, now=None]

t:  97%|████████████████████████

Moviepy - Done !
Moviepy - video ready project_video_out.mp4
Wall time: 24min 26s
